Этот скрипт с описаниями всех шагов создает и формирует базу данных по потоку заявок из сырых .txt файлов. В нулевом блоке необходимо указать ваши параметры:
1. DB_NAME - желаемое имя базы данных
2. PATH - путь к директории, содержащей сырые .txt файлы (можно относительный к этому скрипту)
3. SEPARATOR - разграничитель в полях сырых файлов (ожидается ',', поскольку Биржа предоставляет данные именно в таком формате, но этот факт может быть изменен пользователем).

После всех этих настроек запуск скрипта обеспечивает формирование полной базы данных (пожалуйста, убедитесь, что на вашем диске достаточно места для формирования такой БД: грубая оценка веса всех .txt файлов, подлежащих обработке). 

# 0. Структура

Подключаются необходимые библиотеки, заполняются глобальные переменные (CAPS_LOCK), далее вся логика реализована 
в функции main(). 
Для подробностей можно читать далее (начиная с раздела 1): подготовлено описание не самых очевидных моментов функции main().

In [1]:
from sqlalchemy import create_engine

import numpy as np
import pandas as pd
import psycopg2
import os

In [2]:
DB_NAME = 'orderlogs'  # the name of PostgreSQL Database
QUERY_CREATION = 'CREATE DATABASE "{}" WITH OWNER = postgres TEMPLATE = template0 ENCODING = UTF8'.format(
    DB_NAME)  # generation query
PATH = './data_initial/'  # path to the Orderlog***.txt files
SEPARATOR = ','  # the separator of data columns in the raw data files

In [3]:
def main():
    !echo "SELECT '{QUERY_CREATION}' WHERE NOT EXISTS (SELECT FROM pg_database WHERE datname = '{DB_NAME}') \gexec" | psql
    lst_table_names = get_table_names(PATH)  # get the names for all tables in the dataframe
    creation_data_tables(DB_NAME, lst_table_names)  # creation the SQL tables with fields restrictions
    table_filling(lst_table_names)  # filling the tables with the data from the raw .txt files

# 1. Создание БД 

Развертывается сервер PostgreSQL локально; создается база данных с именем глобальной переменной DB_NAME.

(1 строка функции main() - соответствующий bash скрипт)

## 1.1. Создание таблиц для ордерлогов

Здесь создаем подключение к созданной БД и прописываем необходимые ограничения на поля:
1. NO - номер записи (bigint)
2. SECCODE - тикер инструмента (строка с плавающим числом знаков, до 20, чтобы снизить размер БД)
3. BUYSELL - идентификатор купить / продать (строка с 1 знаком: 'B' или 'S')
4. TIME - время заявки в формате целого числа (bigint)
5. ORDERNO - номер заявки (bigint)
6. ACTION - тип заявки (постановка - 1, снятие - 0, исполнение - 2, smallint, чтобы снизить размер БД)
7. PRICE - цена заявки, числа с плавающей точкой
8. VOLUME - объем заявки (bigint)
9. TRADENO - номер сделки, если таковая произошла (bigint, может быть NULL)
10. TRADEPRICE - цена сделки, если таковая произошла (real, может быть NULL)

Ключом таблицы является уникальный номер записи, "NO".

In [4]:
def creation_data_tables(database_name, lst_table_name):
    '''This function creates in the database with name database_name the series of 
    tables with names from list lst_table_name
    
    :database_name: string
    :lst_table_name: list of strings
    '''
    for table_name in lst_table_name:
        with psycopg2.connect(dbname=DB_NAME,
                          user='postgres',
                          password='',
                          host='localhost',
                          port=5432) as conn:

            cursor = conn.cursor()

            cursor.execute(f'''
            CREATE TABLE IF NOT EXISTS "{table_name}"
                (
                "NO" bigint NOT NULL,
                "SECCODE" varchar(20) COLLATE pg_catalog."default" NOT NULL,
                "BUYSELL" char(1) COLLATE pg_catalog."default" NOT NULL,
                "TIME" bigint NOT NULL,
                "ORDERNO" bigint NOT NULL,
                "ACTION" smallint NOT NULL, 
                "PRICE" real NOT NULL,
                "VOLUME" bigint NOT NULL,
                "TRADENO" bigint,
                "TRADEPRICE" real,
                CONSTRAINT "{table_name}_pkey" PRIMARY KEY ("NO")
                )
            TABLESPACE pg_default;
            ''')

            conn.commit()

### 1.1.1. Получение списка необходимых имен

Из директории, содержащей сырые данные по заявкам (в формате .txt), вид которой примерно следующий: 

- ./data/

    -- OrderLog20190603.txt
    
    -- OrderLog20190606.txt
    
    -- ...
    
    -- .DS_Store

Отбираем только файлы формата .txt, сохраняем идентификатор даты, в формате, например, 20190603, и формируем список из всех доступных дат. 

In [5]:
def get_table_names(path):
    '''This function generates the table names from the initial data files
    
    :path: string that is path to the directory with data files (in .txt format), 
        for example: './data_initial/'
    '''
    names_array = pd.Series(os.listdir(path), dtype=str)
    
    names_array.index = list(map(lambda x: x[-3:], names_array))
    
    names_filtered = sorted(list(map(lambda x: int(x[-12:-4]), names_array['txt'])))

    return names_filtered

## 1.2. Заполнение таблиц из сырых файлов

Далее подхватываем данные в сырых файлах (в .txt) и с помощью консоли добавляем их в сформированные таблицы в базе данных.

In [6]:
def table_filling(lst_table_names):
    '''This function fills the PostgreSQL tables from the lst_table_names
    
    :lst_table_names: the list of table_names in format like 20190606
    '''
    for table_name in lst_table_names:
        print('Start to append table for {}'.format(table_name))
        path = os.path.abspath(os.path.join(PATH, 'OrderLog{}.txt'.format(table_name)))
        !echo 'COPY "{table_name}"' "FROM '{path}' DELIMITERS '{SEPARATOR}' HEADER CSV;" | psql {DB_NAME}
        print('Table {} successfully filled'.format(table_name))

# 2. Запуск main()

Запускаем main() функцию и видим отображение прогресса по заполнению базы данных из всех файлов в указанной директории. 

In [7]:
if __name__ == '__main__':
    main()

CREATE DATABASE
Start to append table for 20190603
COPY 16642888
Table 20190603 successfully filled
Start to append table for 20190604
COPY 13385835
Table 20190604 successfully filled
Start to append table for 20190605
COPY 9761893
Table 20190605 successfully filled
Start to append table for 20190606
COPY 9500628
Table 20190606 successfully filled
Start to append table for 20190607
COPY 8301152
Table 20190607 successfully filled
Start to append table for 20190610
COPY 7807791
Table 20190610 successfully filled
Start to append table for 20190611
COPY 9639997
Table 20190611 successfully filled
Start to append table for 20190613
COPY 9606738
Table 20190613 successfully filled
Start to append table for 20190614
COPY 9250853
Table 20190614 successfully filled
Start to append table for 20190617
COPY 9548751
Table 20190617 successfully filled
Start to append table for 20190618
COPY 9991164
Table 20190618 successfully filled
Start to append table for 20190619
COPY 7973424
Table 20190619 succes